In [0]:
from keras.models impot Sequential
from keras.layers import Dense
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# keras 문법

In [0]:
from keras.models import Sequential
from keras.layers import Dense

In [7]:
model = Sequential()

# model.add(Embedding(voc, hidden_size, input_length))
# model.add(Dense(2, input_dim=4, init="uniform", activation="linear"))    # init: 가중치 초기화 방법    # activation은 linear (활성함수없음) 이 디폴트.

model.add(Dense(4, input_dim=10, init="normal", activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()    # 요약 정보
model.compile()    # 함수 설정
model.fit()    # 학습
model.evaluate()    # 모델 평가
model.predict()    # 예측
model.save()    # 모델 저장
load_model()    # 모델 불러오법

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, input_dim=10, activation="relu", kernel_initializer="normal")`
  
W0816 00:18:45.980287 139979211335552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 00:18:45.994808 139979211335552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0816 00:18:46.020947 139979211335552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 44        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


# 용어 정리

In [8]:
import tensorflow as tf
tf.__version__

'1.14.0'

In [10]:
tf.get_default_graph()    # 기본 그래프의 정보를 출력하는 함수

In [0]:
testGraph = tf.Graph()    # 사용자 그래프 생성
with testGraph.as_default():    # 사용자 그래프에 노드 정의
  x = tf.constant(10)
  y = x + 20

In [12]:
# Session(): 세션객체 생성, 분산처리환경에서 노드를 연결
# run(): 연산 수행 => 결과를 리턴
# close(): 세션 종료

x = tf.constant(5)
y = x + 10
sess = tf.Session()
print(sess.run(x))
print(sess.run(y))
sess.close()

5
15


In [13]:
with tf.Session() as sess:    # 특별히 그래프를 지정하지 않았으므로, 기본 그래프를 사용해서 세션을 실행
  print(sess.run(x))

5


In [15]:
with tf.Session(graph=testGraph) as sess:
  print(sess.run(x))

10


In [0]:
# sess = tf.InteractiveSession()
# x.eval()
# y.eval()

In [19]:
var1 = tf.Variable(5, name="var1")
var2 = tf.add(var1, 5, name="var2")
var1.op.name    # .op.name: 이름 확인할 때 사용

'var1'

In [22]:
# 이름 공간 (name space, 폴더와 비슷) 을 이용한 노드 이름 중복 방지
# tf.name_scope(): 이름 공간 생성

with tf.name_scope("ns1"):
  v3 = tf.Variable(1, name="var3")
  v4 = tf.add(v3, 2, name="var4")

print(v3.op.name)

ns1_1/var3


# 간단하게 만든 예시 1

In [0]:
import numpy as np
from tensorflow.contrib import rnn
import pprint

In [39]:
tf.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
h = [1, 0, 0, 0]    # hello => 4글자
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [40]:
with tf.variable_scope("first_cell") as scope:
  hidden_size = 8
  cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
  print(cell.output_size, cell.state_size)    # output_size = 위쪽으로 출력 = 2, state_size = 오른쪽으로 출력 = 2

  xdata=np.array([[h]], dtype=np.float32)
  outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())
  pp.pprint(_states.eval())

8 8
array([[[-0.08856976,  0.44383818, -0.05266043,  0.2860081 ,
         -0.37913352,  0.09858979,  0.01591983, -0.19578555]]],
      dtype=float32)
array([[-0.08856976,  0.44383818, -0.05266043,  0.2860081 , -0.37913352,
         0.09858979,  0.01591983, -0.19578555]], dtype=float32)


# 간단하게 만든 예시 2

In [41]:
tf.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

with tf.variable_scope("sec_cell") as scope:
  hidden_size = 2
  cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
  xdata=np.array([[h, e, l, l, o]], dtype=np.float32)
  print(xdata.shape)    # (1, 5, 4)    # (batch_size, sequence_length, input_dim)
  pp.pprint(xdata)
  outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.38374177, -0.6554172 ],
        [ 0.56749403, -0.06853331],
        [-0.752503  ,  0.47230798],
        [-0.38278943, -0.11235283],
        [-0.55986804, -0.67451984]]], dtype=float32)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# 간단하게 만든 예시 3

In [43]:
tf.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

with tf.variable_scope("3rd_cell") as scope:
  hidden_size = 2
  cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
  xdata=np.array([[h, e, l, l, o],[e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)
  print(xdata.shape)    # (3, 5, 4)    # (batch_size, sequence_length, input_dim)
  pp.pprint(xdata)
  outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.6676537 , -0.6019233 ],
        [ 0.7936173 , -0.7081999 ],
        [-0.55498505,  0.83853763],
        [-0.6172974 , -0.22932424],
        [ 0.1411274 , -0.6001775 ]],

       [[ 0.54769295, -0.5960476 ],
        [-0.13860485,  0.18727453],
        [-0.55536276,  0.27319095],
        [-0.4686872 , -0.01829517],
        [-0.4055003 ,  0.14699231]],

       [[-0.5411016 ,  0.41371927],
        [-0.51259303, -0.06266409],
        [ 0.67671084, -0.7574146 ],
        [ 0.5755363 ,  0.03330304],
        [-0.67793596,  0.660985  ]]], dtype=float32)


# MVC 분리
M (Model, 시트), V (View, 그래프), C (Controller, 컨트롤러(막대)) 를 나누어 프로그래밍하는 방법론

# 간단하게 만든 예시 4 - LSTM

In [44]:
tf.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

with tf.variable_scope("lstm_3rd_cell") as scope:
  hidden_size = 2
  cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
  xdata=np.array([[h, e, l, l, o],[e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)
  print(xdata.shape)    # (3, 5, 4)    # (batch_size, sequence_length, input_dim)
  pp.pprint(xdata)
  outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
W0816 05:45:52.512214 139979211335552 deprecation.py:323] From <ipython-input-44-41d0cf6eb2eb>:12: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.13632478, -0.0369527 ],
        [-0.04207805, -0.07685976],
        [ 0.07408133, -0.11566778],
        [ 0.15553662, -0.14890121],
        [ 0.03118856, -0.19289055]],

       [[-0.12661386, -0.03583279],
        [-0.2125454 , -0.10352051],
        [-0.05324372, -0.13061532],
        [ 0.0655233 , -0.15213537],
        [ 0.14893332, -0.16977836]],

       [[ 0.10844152, -0.08050708],
        [ 0.17837085, -0.13035168],
        [ 0.01231782, -0.16748656],
        [-0.12931539, -0.16593476],
        [-0.0032582 , -0.15602121]]], dtype=float32)


# 간단하게 만든 예시 5
sequence_length 넣어주기 (근데 안하는게 나음)

In [45]:
tf.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

with tf.variable_scope("lstm_3rd_cell_2") as scope:
  hidden_size = 2
  cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
  xdata=np.array([[h, e, l, l, o],[e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)
  print(xdata.shape)    # (3, 5, 4)    # (batch_size, sequence_length, input_dim)
  pp.pprint(xdata)
  outputs, _states = tf.nn.dynamic_rnn(cell, xdata, sequence_length=[5, 3, 4], dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
W0816 05:51:24.965661 139979211335552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/rnn.py:244: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.07847746,  0.06301965],
        [ 0.03958199, -0.02912069],
        [-0.07801468, -0.08288717],
        [-0.12918392, -0.11627953],
        [-0.11082663, -0.08055113]],

       [[-0.00381683, -0.06085798],
        [-0.0117114 , -0.02309018],
        [-0.09861764, -0.07781125],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.09492068, -0.06818724],
        [-0.13808633, -0.10856909],
        [-0.10568068, -0.11730836],
        [-0.06441989, -0.1162836 ],
        [ 0.        ,  0.        ]]], dtype=float32)


# 간단하게 만든 예시 6

In [46]:
tf.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

with tf.variable_scope("init_lstm_3rd_cell") as scope:
  hidden_size = 2
  batch_size = 3
  cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
  xdata=np.array([[h, e, l, l, o],[e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)
  print(xdata.shape)    # (3, 5, 4)    # (batch_size, sequence_length, input_dim)
  pp.pprint(xdata)
  init_state = cell.zero_state(batch_size, tf.float32)
  outputs, _states = tf.nn.dynamic_rnn(cell, xdata, initial_state=init_state, dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.12252796,  0.09473698],
        [-0.00027119, -0.02567664],
        [-0.09294356,  0.03271559],
        [-0.16470803,  0.0809676 ],
        [-0.2549698 , -0.11496142]],

       [[ 0.09800425, -0.08344842],
        [ 0.05748983, -0.18675798],
        [-0.03832824, -0.11828725],
        [-0.11077959, -0.05261737],
        [-0.16874294,  0.00345491]],

       [[-0.09558548,  0.05843815],
        [-0.1686878 ,  0.10336342],
        [-0.08026518, -0.03298074],
        [ 0.04695508, -0.11198238],
        [-0.05515935, -0.05464489]]], dtype=float32)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# reshape

In [47]:
batch_size = 3
seq_len = 5
in_dim = 3
xdata = np.arange(45).reshape(batch_size, seq_len, in_dim)
print(xdata)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]]

 [[15 16 17]
  [18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]]

 [[30 31 32]
  [33 34 35]
  [36 37 38]
  [39 40 41]
  [42 43 44]]]


# depth 늘리기

In [51]:
with tf.variable_scope("multi_rnn_cell") as scope:
  cell = tf.contrib.rnn.BasicLSTMCell(num_units=5)    # hidden_size = 5
  cell = tf.contrib.rnn.MultiRNNCell([cell]*3)    # layer를 3개로
  outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())

ValueError: ignored

# RNN에서 cost 구하는 과정

In [52]:
ydata = tf.constant([[1, 1, 1]])
prediction1 = tf.constant([[[0.3, 0.7], [0.1, 0.9], [0.3, 0.7]]])
prediction2 = tf.constant([[[0.4, 0.6], [0.4, 0.6], [0.6, 0.4]]])
loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, ydata)
sess.run(tf.global_variables_initializer())
loss1.eval()

TypeError: ignored

# Tensorflow를 이용한 RNN

In [72]:
tf.reset_default_graph()

# 입력 4개 -> 출력 (hidden_size) 2개

# "hihell" => 입력
# "ihello" => 출력

# time_step: 6, input_dim (전체 vocabulary의 사이즈): 5, hidden_size: 5

idx2char = ["h", "i", "e", "l", "o"]
# 학습: hihell -> ihello

# 트레이닝 과정
# hihell -> elhill
# ...
# hihell -> ihehho
# ...
# hihell -> ihello

xdata = [[0 ,1, 0, 2, 3, 3]]    # xdata = "hihell"    # idx2char에서 각 알파벳의 인덱스넘버를 가져옴
x_onehot = [[
             [1, 0, 0, 0, 0],
             [0, 1, 0, 0, 0],
             [1, 0, 0, 0, 0],
             [0, 0, 1, 0, 0],
             [0, 0, 0, 1, 0],
             [0, 0, 0, 1, 0]
]]
ydata = [[1, 0, 2, 3, 3, 4]]    # "ihello"
num_classes = 5
input_dim = 5    # 입력 데이터의 차원 (단어 벡터의 길이)
hidden_size = 5
batch_size = 1
sequence_length = 6    # "hihell"
learning_rate = 0.1

x = tf.placeholder(tf.float32, [None, sequence_length, input_dim])    # shape는 (batch_size, sequence_length, input_dim)
y = tf.placeholder(tf.int32, [None, sequence_length])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)    # num_units는 출력 갯수
ini_state = cell.zero_state(batch_size, tf.float32)    # 초기화

# tf.nn.dynamic_rnn(cell, xdata, initial_state, dtype)
outputs, _states = tf.nn.dynamic_rnn(cell, x, initial_state=ini_state, dtype=tf.float32)
x_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(inputs=x_fc, num_outputs=num_classes)    # outputs를 fully connected 형태로 변환
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])

# tf.contrib.seq2seq.sequence_loss(): cost 구하는 함수
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=weights)    # logit: 예측값, target: 실제값
loss = tf.reduce_mean(sequence_loss)


prediction = tf.argmax(outputs, axis=2)    # outputs는 3차원인데 그 중에 가장 안쪽에 있는 num_classes에 대해 argmax를 구해야 하므로 axis=2
# prediction은 2차원이 된다 (3차원에서 마지막꺼만 argmax 했으므로)

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(101):
    lv, _ = sess.run([loss, train], feed_dict={x:x_onehot, y:ydata})
    res = sess.run(prediction, feed_dict={x:x_onehot})
    print("step: ", i, ", cost: ", lv, ", prediction: ", res, ", true Y: ", ydata)
    resStr = [idx2char[c] for c in np.squeeze(res)]
    print("prediction str: ", "".join(resStr))

step:  0 , cost:  1.620883 , prediction:  [[1 0 2 0 1 1]] , true Y:  [[1, 0, 2, 3, 3, 4]]
prediction str:  ihehii
step:  1 , cost:  1.6065127 , prediction:  [[2 2 2 0 1 1]] , true Y:  [[1, 0, 2, 3, 3, 4]]
prediction str:  eeehii
step:  2 , cost:  1.577199 , prediction:  [[2 2 2 2 2 2]] , true Y:  [[1, 0, 2, 3, 3, 4]]
prediction str:  eeeeee
step:  3 , cost:  1.5562338 , prediction:  [[2 2 2 2 2 2]] , true Y:  [[1, 0, 2, 3, 3, 4]]
prediction str:  eeeeee
step:  4 , cost:  1.5205675 , prediction:  [[2 2 2 3 3 3]] , true Y:  [[1, 0, 2, 3, 3, 4]]
prediction str:  eeelll
step:  5 , cost:  1.4520413 , prediction:  [[2 2 2 3 3 3]] , true Y:  [[1, 0, 2, 3, 3, 4]]
prediction str:  eeelll
step:  6 , cost:  1.3810911 , prediction:  [[2 2 2 3 3 3]] , true Y:  [[1, 0, 2, 3, 3, 4]]
prediction str:  eeelll
step:  7 , cost:  1.2979809 , prediction:  [[2 3 2 3 3 3]] , true Y:  [[1, 0, 2, 3, 3, 4]]
prediction str:  elelll
step:  8 , cost:  1.214488 , prediction:  [[2 3 2 3 3 3]] , true Y:  [[1, 0, 2, 3,

# Squeeze 함수

In [65]:
test = [[3, 1, 0, 2, 1]]
np.squeeze(test)    # 차원을 내려주는 함수

array([3, 1, 0, 2, 1])

In [67]:
test2 = [idx2char[c] for c in np.squeeze(test)]
test2

['l', 'i', 'h', 'e', 'i']